In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Set the display option to a large value to prevent text wrapping
pd.set_option('display.max_colwidth', None)

#### 1. Read all CEO sample files
##### download your CEO sample files from collect.earth and upload to SEPAL using FileZilla. add as many files as you have projects

In [2]:
ceo1 = pd.read_csv('/home/sepal-user/COCAFORI/ceo-COCAFORI-validation-1-couverture-du-sol-2020-sample-data.csv')
ceo2 = pd.read_csv('/home/sepal-user/COCAFORI/ceo-COCAFORI-validation-2-couverture-du-sol-2020-sample-data.csv')
ceo3 = pd.read_csv('/home/sepal-user/COCAFORI/ceo-COCAFORI-validation-3-couverture-du-sol-2020-sample-data.csv')
ceo4 = pd.read_csv('/home/sepal-user/COCAFORI/ceo-COCAFORI-validation-4-couverture-du-sol-2020-sample-data.csv')
ceo5 = pd.read_csv('/home/sepal-user/COCAFORI/ceo-COCAFORI-validation-5-couverture-du-sol-2020-sample-data.csv')
ceo6 = pd.read_csv('/home/sepal-user/COCAFORI/ceo-COCAFORI-validation-6-couverture-du-sol-2020-sample-data.csv')

In [3]:
# output file cleaned CEO validated data file with select columns
ceo_data_out =  '/home/sepal-user/COCAFORI/CMR_LC_validated.csv'

In [4]:
ceo_files = [ceo1,ceo2,ceo3,ceo4,ceo5,ceo6]

### end of parameters

In [5]:
column_names = ceo1.columns.tolist()
# Print the list of column names
print(column_names)

['plotid', 'sampleid', 'sample_internal_id', 'lon', 'lat', 'email', 'flagged', 'collection_time', 'analysis_duration', 'imagery_title', 'imagery_attributions', 'sample_geom', 'pl_index', 'pl_jrcmap', 'pl_tcc', 'pl_classification', 'Type GIEC', 'Type de terre forestière', 'Type de terres cultivées', 'Type de terres humides', 'Commentaires']


##### clean files

In [6]:
# List to store the modified DataFrames
modified_ceo_files = []

# Initialize collection counter
collection_counter = 1

for i, ceo in enumerate(ceo_files):
       
    # Sort the DataFrame by 'date' in descending order
    ceo.sort_values(by='collection_time', ascending=True, inplace=True)

    # Remove duplicates keeping the last occurrence (the most recent date)
    ceo.drop_duplicates(subset='plotid', keep='first', inplace=True)

    # Reset the index
    ceo.reset_index(drop=True, inplace=True)
    
    # Add 'collection' column with an increasing number
    ceo['collection'] = collection_counter
    
    # Add 'interpreted' column with 1 when 'email' is not null, otherwise 0
    ceo['interpreted'] = np.where(ceo['email'].notna(), 1, 0)
    
    # Calculate total count for each 'interpreted' value
    interpreted_counts = ceo['interpreted'].value_counts().to_dict()
    
    # Append the modified DataFrame to the list
    modified_ceo_files.append(ceo)
    
    # Print count for the current DataFrame
    print(f"ceo {i + 1} # of validated points: {interpreted_counts.get(1, 0)}, # of not validated points: {interpreted_counts.get(0, 0)}")
    
    # Increment the collection counter
    collection_counter += 1

# Concatenate the modified DataFrames into one merged DataFrame
merged_ceo = pd.concat(modified_ceo_files, ignore_index=True)


# Specify the columns you want to keep in the merged DataFrame
columns_to_keep = ['plotid', 'sampleid', 'sample_internal_id', 'lon', 'lat', 'flagged', 
                   'collection_time', 'sample_geom', 
                   'pl_index', 'pl_jrcmap', 'pl_tcc', 'Type GIEC', 'Type de terre forestière', 
                   'Type de terres cultivées', 'Type de terres humides', 'collection', 'interpreted']

# Select only the specified columns
merged_ceo = merged_ceo[columns_to_keep]

# Create a dictionary to map old column names to new column names
column_mapping = {
    'Type GIEC': 'IPCC_Ref', 
    "Type de terre forestière": 'FType_2020_Ref',
    "Type de terres cultivées": 'AgType_2020_Ref', 
    'Type de terres humides': 'HumType_2020_Ref', 

}

# Use the rename() method to rename the columns
merged_ceo = merged_ceo.rename(columns=column_mapping)

ceo 1 # of validated points: 576, # of not validated points: 0
ceo 2 # of validated points: 801, # of not validated points: 0
ceo 3 # of validated points: 500, # of not validated points: 0
ceo 4 # of validated points: 130, # of not validated points: 0
ceo 5 # of validated points: 143, # of not validated points: 0
ceo 6 # of validated points: 267, # of not validated points: 0


In [7]:
merged_ceo.head()

,plotid,sampleid,sample_internal_id,lon,lat,flagged,collection_time,sample_geom,pl_index,pl_jrcmap,pl_tcc,IPCC_Ref,FType_2020_Ref,AgType_2020_Ref,HumType_2020_Ref,collection,interpreted
0,4,4,760021835,14.298574,7.776715,False,2024-07-12 11:10,POINT(14.29857421 7.776715415),361,1.0,90.0,Terres Forestières,Forêt Claire (30-60% TCC) sans perturbation,NaN,NaN,1,1
1,2,2,760022043,13.156815,9.444887,False,2024-07-12 11:10,POINT(13.15681548 9.444886897),569,1.0,80.0,Terres Herbacées,NaN,NaN,NaN,1,1
2,6,6,760022039,13.759585,8.647183,False,2024-07-12 11:11,POINT(13.75958504 8.647182925),565,1.0,70.0,Terres Herbacées,NaN,NaN,NaN,1,1
3,7,7,760021989,14.740545,11.144499,False,2024-07-12 11:11,POINT(14.74054533 11.14449941),515,1.0,60.0,Terres Herbacées,NaN,NaN,NaN,1,1
4,8,8,760021562,13.101120,4.402643,False,2024-07-12 11:12,POINT(13.10111994 4.402643207),88,1.0,100.0,Terres Forestières,Forêt Dense (>60% TCC) sans perturbation,NaN,NaN,1,1


In [8]:
len(merged_ceo)

2417

In [9]:
pd.pivot_table(merged_ceo,values='plotid',index=['IPCC_Ref'],columns=['interpreted'],aggfunc="count")

interpreted,1
IPCC_Ref,
Autres Terres,13
Surfaces Humaines,80
Terres Cultivées,171
Terres Forestières,1368
Terres Herbacées,111
Terres Herbacées et Savanes,589
Terres Humides,85


In [10]:
pd.pivot_table(merged_ceo,values='plotid',index=['FType_2020_Ref'],columns=['interpreted'],aggfunc="count")

interpreted,1
FType_2020_Ref,
Forêt Claire (30-60% TCC) sans perturbation,218
Forêt Dense (>60% TCC) sans perturbation,797
Forêt Secondaire/Dégradée avec perturbation depuis 2000,146
Mangrove,19
Savane Boisée (10-30% TCC) sans perturbation,188


In [11]:
pd.pivot_table(merged_ceo,values='plotid',index=['AgType_2020_Ref'],columns=['interpreted'],aggfunc="count")

interpreted,1
AgType_2020_Ref,
"Agriculure (cacao, huile de palme ou autre)",156
Plantation forestière,15


In [12]:
pd.pivot_table(merged_ceo,values='plotid',index=['HumType_2020_Ref'],columns=['interpreted'],aggfunc="count")

interpreted,1
HumType_2020_Ref,
Eau,39
Marécages,46


In [13]:
def y(x):
  if x['IPCC_Ref'] == 'Terres Forestières': return 10
  elif x['IPCC_Ref'] == 'Terres Cultivées' : return 20
  elif x['IPCC_Ref'] == 'Terres Herbacées' or x['IPCC_Ref'] == 'Terres Herbacées et Savanes' : return 30
  elif x['IPCC_Ref'] == 'Terres Humides' : return 40
  elif x['IPCC_Ref'] == 'Surfaces Humaines' : return 50
  elif x['IPCC_Ref'] == 'Autres Terres' : return 60
  else: return 0

merged_ceo['IPCC_2020_Code_Ref'] = merged_ceo.apply(y, axis=1)

annual_counts = merged_ceo['IPCC_2020_Code_Ref'].value_counts()
print(annual_counts)

IPCC_2020_Code_Ref
10    1368
30     700
20     171
40      85
50      80
60      13
Name: count, dtype: int64


In [14]:
def y(x):
  if x['IPCC_2020_Code_Ref'] == 10: return 1
  else: return 0

merged_ceo['FNF_2020_Ref'] = merged_ceo.apply(y, axis=1)

annual_counts = merged_ceo['FNF_2020_Ref'].value_counts()
print(annual_counts)

FNF_2020_Ref
1    1368
0    1049
Name: count, dtype: int64


In [15]:
def y(x):
  if x['FType_2020_Ref'] == 'Forêt Dense (>60% TCC) sans perturbation': return 11
  elif x['FType_2020_Ref'] == 'Forêt Claire (30-60% TCC) sans perturbation' : return 12
  elif x['FType_2020_Ref'] == 'Savane Boisée (10-30% TCC) sans perturbation' : return 13
  elif x['FType_2020_Ref'] == 'Mangrove ' : return 14
  elif x['FType_2020_Ref'] == 'Forêt Secondaire/Dégradée avec perturbation depuis 2000' : return 15
  else: return 0

merged_ceo['FType_2020_Code_Ref'] = merged_ceo.apply(y, axis=1)

annual_counts = merged_ceo['FType_2020_Code_Ref'].value_counts()
print(annual_counts)

FType_2020_Code_Ref
0     1049
11     797
12     218
13     188
15     146
14      19
Name: count, dtype: int64


In [16]:
def y(x):
  if x['AgType_2020_Ref'] == 'Agriculure (cacao, huile de palme ou autre)': return 22
  elif x['AgType_2020_Ref'] == 'Plantation forestière' : return 21
  else: return 0

merged_ceo['AgType_2020_Code_Ref'] = merged_ceo.apply(y, axis=1)

annual_counts = merged_ceo['AgType_2020_Code_Ref'].value_counts()
print(annual_counts)

AgType_2020_Code_Ref
0     2246
22     156
21      15
Name: count, dtype: int64


In [17]:
def y(x):
  if x['HumType_2020_Ref'] == 'Eau': return 42
  elif x['HumType_2020_Ref'] == 'Marécages' : return 41
  else: return 0

merged_ceo['HumType_2020_Code_Ref'] = merged_ceo.apply(y, axis=1)

annual_counts = merged_ceo['HumType_2020_Code_Ref'].value_counts()
print(annual_counts)

HumType_2020_Code_Ref
0     2332
41      46
42      39
Name: count, dtype: int64


In [18]:
# Define the function to set Class_Code based on conditions
def set_class_code(row):
    if row['FType_2020_Code_Ref'] != 0:
        return row['FType_2020_Code_Ref']
    elif row['AgType_2020_Code_Ref'] != 0:
        return row['AgType_2020_Code_Ref']
    elif row['HumType_2020_Code_Ref'] != 0:
        return row['HumType_2020_Code_Ref']
    else:
        return row['IPCC_2020_Code_Ref']

# Apply the function to each row
merged_ceo['LC_2020_Code_Ref'] = merged_ceo.apply(set_class_code, axis=1)

In [19]:
pd.pivot_table(merged_ceo,values='plotid',index=['LC_2020_Code_Ref'],columns=['interpreted'],aggfunc="count")

interpreted,1
LC_2020_Code_Ref,
11,797
12,218
13,188
14,19
15,146
21,15
22,156
30,700
41,46


In [20]:
# export
merged_ceo.index.name = 'index'
merged_ceo.to_csv(ceo_data_out,index=True)